<h1><center><font color='blue'>MODELOS PRE-ENTRENADOS</font></center></h1>

## Objetivo


Veremos cómo aprovechar modelos pre-entrenados para construir clasificadores en lugar de hacer un modelo desde cero.


## Tabla de contenido

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 
    
1. <a href="#item31">Importando librerías y paquetes</a>
2. <a href="#item32">Descargando los datos</a>  
3. <a href="#item33">Definiendo constantes globales</a>  
4. <a href="#item34">Construyendo instanciasImageDataGenerator</a>  
5. <a href="#item35">Compilando y ajustando el modelo</a>

</font>
    
</div>


<a id='item31'></a>


## Importando librerías y paquetes


Importamos el módulo ImageDataGenerator ya que lo aprovecharemos para entrenar nuestro modelo en lotes.


In [1]:
from keras.preprocessing.image import ImageDataGenerator

Importamos Keras


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Aprovecharemos el modelo ResNet50 para construir nuestro clasificador.


In [3]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

<a id='item32'></a>


## Descargando los datos


In [4]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

import urllib.request
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip'
filename = 'concrete_data_week3.zip'
urllib.request.urlretrieve(url, filename) 


('concrete_data_week3.zip', <http.client.HTTPMessage at 0x1b02dbaee20>)

Ahora verá en la carpeta apropiada el archivo _concrete_data_week3.zip_. Lo descomprimimos:


In [5]:
#!unzip concrete_data_week3.zip

import zipfile
with zipfile.ZipFile('concrete_data_week3.zip', 'r') as zip_ref:
    zip_ref.extractall()


Ahora verá la carpeta _concrete_data_week3_. En ella hay 2 carpetas: _train_ y _valid_. Cada una contiene 2 subcarpetas: _positive_ y _negative_ donde negatve contiene las imágenes sin grietas y positive las que sí contienen grietas.


<a id='item33'></a>


## Definiendo constantes globales


Definiremos constantes que usaremos en el resto de la notebook:

1.  Estamos tratando con 2 clases, por lo que _num_classes_ es 2. 
2.  El modelo ResNet50 fue construido y entrenado usando imágenes de tamaño (224 x 224). Por tanto debemos cambiar el tamaño de las imágenes de (227x227) a (224x224).
3. Para entrenar y validar el modelo usaremos lotes de 100 imágenes.


In [6]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>


## Construyendo instancias ImageDataGenerator


Para instanciar una instancia de ImageDataGenerator estableceremos el argumento **preprocessing_function** a _preprocess_input_ que importamos de **keras.applications.resnet50** para preprocesar nuestras imágenes de la misma forma que fueron pre-procesadas cuando se entrenó el modelo ResNet50.


In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Usaremos el método  _flow_from_directory_ para obtener las imágenes entrenadas como sigue:


In [8]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


Repetimos pero ahora con las imágenes de validación:


In [10]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


Found 10001 images belonging to 2 classes.


<a id='item35'></a>


## Construir, compilar y ajustar el modelo


Comenzamos construyendo el modelo. Utilizaremos el modelo Sequential de Keras.


In [11]:
model = Sequential()

Agregamos el modelo ResNet50 pre-entrenado a nuestro modelo. Sin embargo, tenga en cuenta que no queremos incluir la capa superior o la capa de salida del modelo previamente entrenado, queremos definir nuestra propia capa de salida y entrenarla para que esté optimizada para nuestro conjunto de imágenes. Para dejar fuera la capa de salida del modelo pre-entrenado, estableceremos el argumento _include_top_ en **False**.

In [12]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

94773248/94765736 [==============================] - 24s 0us/step


Luego, definimos nuestra capa de salida como una capa **Dense**, que consiste de 2 nodos y usa la función **Softmax** como función de activación.

In [13]:
model.add(Dense(num_classes, activation='softmax'))

Puede acceder a las capas del modelo usando el atributo _layers_ de nuestro objeto modelo.


In [14]:
model.layers

Puede ver que nuestro modelo está compuesto por 2 capas. La primera es la perteneciente al ResNet50, y la segunda es la capa Dense.


Puede acceder a las capas del ResNet50 ejecutando:


In [15]:
model.layers[0].layers

Como el modelo ResNet50 ya ha sido entrenado, queremos decirle a nuestro modelo que no se moleste en entrenar la parte del ResNet50; para ello ejecutamos lo siguiente:


In [16]:
model.layers[0].trainable = False

Usando el atributo _summary_ del modelo podemos ver cuántos parámetros necesitamos optimizar para entrenar la capa de salida.


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Compilamos el modelo utilizando el optimizador **adam**


In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Antes de comenzar el proceso de entrenamiento, con ImageDataGenerator, necesitamos definir cuántos pasos componen un epoch. Típicamente, es el número de imágenes dividido por el tamaño del lote. Por tanto, definimos nuestros pasos por epoch como sigue:


In [19]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Estamos listos para entrenar el modelo. A diferencia del deep learning convencional, donde los datos no se transmiten desde un directorio, con ImageDataGenerator donde los datos se aumentan en lotes, usamos el método **fit_generator**.


In [20]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 2009s 7s/step - loss: 0.0259 - accuracy: 0.9925 - val_loss: 0.0066 - val_accuracy: 0.9985
Epoch 2/2
301/301 [==============================] - 2015s 7s/step - loss: 0.0054 - accuracy: 0.9987 - val_loss: 0.0043 - val_accuracy: 0.9991


Ahora que el modelo fue entrenado, puede usarlo para comenzar a clasificar imágenes.


Como el entrenamiento puede demorar mucho con modelos de deep learning, siempre es una buena idea guardar el modelo una vez se completó el entrenamiento si cree que lo utilizará luego. Guardémoslo entonces:


In [21]:
model.save('classifier_resnet_model.h5')

Se generó así el archivo _classifier_resnet_model.h5_.
